# <center>Laboratorium Analiza i bazy danych </center>

## <center>Łączenie tabel, podzapytania i funkcje agregujące</center>

## Przykładowe tabele obrazujące łączenie

Do zobrazowania operacji łączenia zostaną użyte tabele:

```sql
CREATE TABLE shape_a (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
 
CREATE TABLE shape_b (
    id INT PRIMARY KEY,
    shape VARCHAR (100) NOT NULL
);
```
 
Polecenie CREATE TABLE tworzy tabelę o zadanej nazwie i strukturze. Ogólna postać to:
```sql
CREATE TABLE tab_name (
    col_name1 data_type constrain,
    col_name1 data_type constrain,
    ...
);
```
Należy uzupełnić ją danymi:
```sql
INSERT INTO shape_a (id, shape)
VALUES
    (1, 'Trójkąt'),
    (2, 'Kwadrat'),
    (3, 'Deltoid'),
    (4, 'Traper');
 
INSERT INTO shape_b (id, shape)
VALUES
    (1, 'Kwadrat'),
    (2, 'Trójkąt'),
    (3, 'Romb'),
    (4, 'Równoległobok');
```
Komenda INSERT INTO pozwala na dodanie do tabeli rekordów. Ogólna postać to:

```sql
INSERT INTO tab_name (col1_name, col2_name2, ...) 
VALUES
    (val1_col1, val2_col2),
    (val2_col1, val2_col2),
    ...
```

## Inner join 

Jest to podstawowy rodzaj złączenie. Ten sposób złączenia wybiera  te wiersze, dla których warunek złączenia jest spełniony. W żadnej z łączonych tabel kolumna użyta do łączenia nie może mieć wartości NULL. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
INNER JOIN shape_b b ON a.shape = b.shape;
```
W zapytaniu powyżej użyto *aliasów* nazw tabel i column wynikowych, jest to szczególnie przydatne przy długich nazwach tabel i wprowadza czytelność w zapytaniu.

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|

## OUTER JOIN

Istnieją trzy rodzaje złączeń OUTER:
- LEFT OUTER JOIN,
- RIGHT OUTER JOIN,
- FULL OUTER JOIN.

### LEFT OUTER JOIN

Ten rodzaj złączenie zwróci wszystkie rekordy z lewej tablicy i dopasuje do nich rekordy z prawej tablicy które spełniją zadany warunek złączenia. Jeżeli w prawej tablicy nie występują rekordy spełnijące warunek złączenia z lewą w ich miejscu pojawią się wartości NULL.

#### Przykład 1:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
LEFT JOIN shape_b b ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid|NULL|NULL|
|4|Traper|NULL|NULL|

#### Przykład 2:
```sql
SELECT
    b.id id_b,
    b.shape shape_b,
    a.id id_a,
    a.shape shape_a   
FROM
    shape_b b
LEFT JOIN shape_a a ON a.shape = b.shape;
```
#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Kwadrat|2|Kwadrat|
|2|Trójkąt|1|Trójkąt|
|3|Romb|NULL|NULL|
|4|Równoległobok|NULL|NULL|

### RIGHT OUTER JOIN

Działa jak left outer join z tym, że prawa tablica w zapytaniu jest brana w całości.

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
RIGHT JOIN shape_b b ON a.shape = b.shape;
```

#### Wynik:
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|2|Kwadrat|1|Kwadrat|
|1|Trójkąt|2|Trójkąt|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|


### FULL OUTER JOIN

Jest złączeniem które zwraca:
- wiersze dla których warunek złączenia jest spełniony,
- wiersze z lewej tabeli dla których nie ma odpowiedników w prawej,
- wiersze z prawej tabeli dla których nie ma odpowiedników w lewej. 

#### Przykład:
```sql
SELECT
    a.id id_a,
    a.shape shape_a,
    b.id id_b,
    b.shape shape_b
FROM
    shape_a a
FULL JOIN shape_b b ON a.shape = b.shape;
```
|id_a|shape_a|id_b|shape_b|
|-|-|-|-|
|1|Trójkąt|2|Trójkąt|
|2|Kwadrat|1|Kwadrat|
|3|Deltoid"|NULL|NULL|
|4|Traper|NULL|NULL|
|NULL|NULL|3|Romb|
|NULL|NULL|4|Równoległobok|

## Podzapytania

Podzapytanie zagnieżdżone SELECT znajduje się wewnątrz zewnętrznego zapytania SELECT, np. po klauzuli WHERE, HAVING lub FROM. W przypadku tego rodzaju zapytań w pierwszej kolejności wykonywane są wewnętrzne zapytania SELECT, a ich wynik jest wykorzystywany do zewnętrznego zapytania SELECT. Stąd łatwo zuważyć, że mogą one służyć do poprawy wydajności obsługi zapytania. Należy dobierać podzapytania tak by najbardziej zagnieżdżone podzapytanie zawierało najmniejszy zbiór poszukiwań. 

#### Przykład:
Jeżeli chcemy znaleźć w bazie informację o tytułach filmów zwróconych w zadanym okresie możemy wykonać następujące zapytanie:
```sql
SELECT
   film_id,
   title
FROM
   film
WHERE
   film_id IN (
      SELECT
         inventory.film_id
      FROM
         rental
      INNER JOIN inventory ON inventory.inventory_id = rental.inventory_id
      WHERE
         return_date BETWEEN '2005-05-29'
      AND '2005-05-30'
   );
```

#### Wynik
|film_id|title|
|-|-|
|307|Fellowship Autumn|
|255|Driving Polish|
|388|Gunfight Moon|
|130|Celebrity Horn|
|563|Massacre Usual|
|397|Hanky October|
|...|...|

### Używanie podzapytań

Pod zapytania mogą być używane w :
- SELECT,
- UPDATE,
- DELETE,
- Funkcjach agregujących,
- Do definiowania tabel tymczasowych.

Używając podzapytań zapytania SQL szybko mogą stać się mało czytelne. Przez co będą trudne w zrozumieniu i późniejszym utrzymaniu. W celu analizy zapytań można użyć klauzuli __EXPLAIN__, która przeanalizuje zapytanie. Klauzula ta może służyć również do porównywania wydajności zapytań

#### Przykład:
```sql
EXPLAIN SELECT
   *
FROM
   film
```

## Funkcje agregujące

Funkcje agregujące wykonują obliczenia na zestawie wierszy i zwracają pojedynczy wiersz. PostgreSQL udostępnia wszystkie standardowe funkcje agregujące SQL w następujący sposób:
- AVG () - zwraca średnią wartość.
- COUNT () - zwraca liczbę wartości.
- MAX () - zwraca maksymalną wartość.
- MIN () - zwraca minimalną wartość.
- SUM () - zwraca sumę wszystkich lub różnych wartości.

Pełna lista funkcji agregującej: https://www.postgresql.org/docs/9.5/functions-aggregate.html

Często używamy funkcji agregujących z klauzulą GROUP BY w instrukcji SELECT. W tych przypadkach klauzula GROUP BY dzieli zestaw wyników na grupy wierszy i funkcja agregująca wykonuje obliczenia dla każdej grupy, np. maksimum, minimum, średnia itp. Funkcji agregujących można używać funkcji agregujących jako wyrażeń tylko w następujących klauzulach: SELECT i HAVING.

### GROUP BY
Klauzula GROUP BY dzieli wiersze zwrócone z instrukcji SELECT na grupy. Dla  każdej grupy można zastosować funkcję agregującą, np. SUM aby obliczyć sumę pozycji lub
COUNT aby uzyskać liczbę elementów w grupach.

Poniższa instrukcja ilustruje składnię klauzuli GROUP BY:
```sql
SELECT 
    column_1, 
    aggregate_function(column_2)
FROM 
    tbl_name
GROUP BY 
    column_1;
```
Klauzula GROUP BY musi pojawić się zaraz po klauzuli FROM lub WHERE, n0astępnie GROUP BY zawiera listę  kolumna oddzielonych przecinkami. 

### HAVING
Często używamy klauzuli HAVING w połączeniu z klauzulą GROUP BY do filtrowania wierszy grup
które nie spełniają określonego warunku.

Poniższa instrukcja ilustruje typową składnię klauzuli HAVING:
```sql
SELECT
    column_1,
    aggregate_function (column_2)
FROM
    tbl_name
GROUP BY
    column_1
HAVING
    condition;
```
Klauzula HAVING ustawia warunek dla wierszy grup utworzonych przez klauzulę GROUP BY.  

Klauzula GROUP BY ma zastosowanie, podczas gdy klauzula WHERE określa wcześniej warunki dla poszczególnych wierszy.

## Zadania wprowadzające
Wykonaj zapytania przy użyciu DBMS:  
  
1. Znajdź listę wszystkich filmów o tej samej długości.
2. Znajdź wszystkich klientów mieszkających w tym samym mieście.
3. Oblicz średni koszt wypożyczenia wszystkich filmów.
4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.
5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.
6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.
7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.
8. Oblicz średnią wartość wypożyczenia filmu.
9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.
10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.
11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [1]:
import numpy as np
import pickle

import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd

from typing import Union, List, Tuple

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020');

In [2]:
import warnings
warnings.filterwarnings('ignore')

1. Lista wszystkich filmów o tej samej długości

In [3]:
df = pd.read_sql("select count(film_id), length from film group by length", con=connection)
df

,count,length
0,4,116
1,8,184
2,6,87
3,7,71
4,5,68
...,...,...
135,7,148
136,7,129
137,6,130
138,7,143


2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [4]:
df = pd.read_sql("select count(customer_id), city.city from customer\n"
                 "inner join address on customer.address_id = address.address_id\n"
                 "inner join city on address.city_id = city.city_id\n"
                 "group by city.city", con=connection)
df

,count,city
0,1,Southport
1,1,Taguig
2,1,Tokat
3,1,Atlixco
4,1,Mukateve
...,...,...
592,1,Warren
593,1,Pingxiang
594,1,Greensboro
595,1,Ocumare del Tuy


3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [5]:
df = pd.read_sql("select avg(rental_rate) from film", con=connection)
df

,avg
0,2.98


4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [6]:
df = pd.read_sql("select category.name, count(film_category.film_id) from category\n"
                 "inner join film_category on category.category_id = film_category.category_id\n"
                 "inner join film on film_category.film_id = film.film_id group by category.name", con=connection)
df

,name,count
0,Sports,74
1,Classics,57
2,New,63
3,Family,69
4,Comedy,58
5,Animation,66
6,Travel,57
7,Music,51
8,Drama,62
9,Horror,56


5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [7]:
df = pd.read_sql("select country.country, count(customer.customer_id) from country\n"
                 "inner join city on country.country_id = city.country_id\n"
                 "inner join address on city.city_id = address.city_id\n"
                 "inner join customer on address.address_id = customer.address_id group by country.country", con=connection)
df

,country,count
0,Bangladesh,3
1,Indonesia,14
2,Venezuela,7
3,Cameroon,2
4,Czech Republic,1
...,...,...
103,Tanzania,3
104,Poland,8
105,Greenland,1
106,French Guiana,1


6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [8]:
df = pd.read_sql("select store.store_id, store.manager_staff_id, store.last_update, store.address_id, count(payment.customer_id) from store\n"
                 "inner join staff on staff.store_id = store.store_id\n"
                 "inner join payment on payment.staff_id = staff.staff_id\n"
                 "inner join customer on customer.customer_id = payment.customer_id\n"
                 "group by store.store_id having count(payment.customer_id) between 100 and 300", con=connection)
df

,store_id,manager_staff_id,last_update,address_id,count


7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [9]:
df = pd.read_sql("select sum(film.length), customer.customer_id, customer.first_name, customer.last_name from customer\n"
                 "inner join rental on rental.customer_id = customer.customer_id\n"
                 "inner join inventory on inventory.inventory_id = rental.inventory_id\n"
                 "inner join film on inventory.film_id = film.film_id\n"
                 "group by customer.customer_id having sum(film.length) > 200", con=connection)
df

,sum,customer_id,first_name,last_name
0,3446,87,Wanda,Patterson
1,2649,184,Vivian,Ruiz
2,2750,477,Dan,Paine
3,3606,273,Priscilla,Lowe
4,3309,550,Guy,Brownlee
...,...,...,...,...
594,2119,449,Oscar,Aquino
595,3736,520,Mitchell,Westmoreland
596,3515,64,Judith,Cox
597,2756,55,Doris,Reed


8. Oblicz średnią wartość wypożyczenia filmu.

In [10]:
df = pd.read_sql("select avg(rental_rate) from film", con=connection)
df

,avg
0,2.98


9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [11]:
df = pd.read_sql("select category.name, avg(film.length) from category\n"
                 "inner join film_category on category.category_id = film_category.category_id\n"
                 "inner join film on film.film_id = film_category.film_id group by category.name", con=connection)
df 

,name,avg
0,Sports,128.202703
1,Classics,111.666667
2,New,111.126984
3,Family,114.782609
4,Comedy,115.827586
5,Animation,111.015152
6,Travel,113.315789
7,Music,113.647059
8,Drama,120.838710
9,Horror,112.482143


10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [12]:
df = pd.read_sql("select category.name, film.title, film.length from category\n"
                 "inner join film_category on category.category_id = film_category.category_id\n"
                 "inner join film on film.film_id = film_category.film_id \n"
                 "where (category.name, film.length) in\n"
                 "(select category.name, max(film.length) from category\n"
                 "inner join film_category on category.category_id = film_category.category_id\n"
                 "inner join film on film.film_id = film_category.film_id\n"
                 "group by category.name)", con=connection)
df 

,name,title,length
0,Horror,Analyze Hoosiers,181
1,Games,Chicago North,185
2,Classics,Conspiracy Spirit,184
3,Comedy,Control Anthem,185
4,Foreign,Crystal Breaking,184
5,Action,Darn Forrester,185
6,New,Frontier Cabin,183
7,Children,Fury Murder,178
8,Animation,Gangs Pride,185
9,Music,Home Pity,185


11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [13]:
df = pd.read_sql("select film.title, film.length from film\n"
                 "where film.length in (select max(film.length) from film)", con=connection)
df

,title,length
0,Chicago North,185
1,Control Anthem,185
2,Darn Forrester,185
3,Gangs Pride,185
4,Home Pity,185
5,Muscle Bright,185
6,Pond Seattle,185
7,Soldiers Evolution,185
8,Sweet Brotherhood,185
9,Worst Banger,185


## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [14]:
import main

In [15]:
main.film_in_category(2)

,title,languge,category
0,Alter Victory,English,Animation
1,Anaconda Confessions,English,Animation
2,Argonauts Town,English,Animation
3,Bikini Borrowers,English,Animation
4,Blackout Private,English,Animation
...,...,...,...
61,Tracy Cider,English,Animation
62,Turn Star,English,Animation
63,Wait Cider,English,Animation
64,Watch Tracy,English,Animation


In [16]:
main.number_films_in_category(5)

,category,count
0,Comedy,58


In [17]:
main.number_film_by_length(120, 150)

,length,count
0,120,9
1,121,8
2,122,11
3,123,7
4,124,4
5,125,7
6,126,9
7,127,5
8,128,7
9,129,7


In [18]:
main.client_from_city('Athenai')

,city,first_name,last_name
0,Athenai,Linda,Williams


In [19]:
main.avg_amount_by_length(120)

,length,avg
0,120,4.931176


In [20]:
main.client_by_sum_length(4500)

,first_name,last_name,sum
0,Diane,Collins,4514
1,Elizabeth,Brown,4531
2,Justin,Ngo,4550
3,Sherry,Marshall,4582
4,Marion,Snyder,4599
5,Russell,Brinson,4640
6,Marcia,Dean,4734
7,Wesley,Bull,4808
8,Clara,Shaw,4808
9,Tammy,Sanders,5065


In [21]:
print(main.category_statistic_length('Action'))

  category         avg   sum  min  max
0   Action  111.609375  7143   47  185
